# Behind the scenes: Computing with kernels, kernel transformations and selection methods

This notebooks gives examples on how to use our implementations of base kernels, kernel transformations and selection methods. These are implemented in the `bmdal` subfolder. If you just want to use our BMDAL methods, please have a look at the notebook on using BMDAL, where we show how to use the `bmdal.algorithms.select_batch()` function to perform BMDAL.

The following examples may be useful if you wish to design your own BMDAL algorithm using our components, use our implemented components for some other purpose (e.g., uncertainty quantification), or implement your own components. For additional examples of usage, you may want to look at the source code files in the `bmdal` folder, especially `bmdal/algorithms.py` and `bmdal/selection.py`.

We will first change the working directory from the examples subfolder to the main folder, which is required for the imports to work correctly.

In [1]:
import os
os.chdir('..')   # change directory inside the notebook to the main directory

When using a GPU that supports TF32 matrix multiplication, we need to disable it to avoid numerical problems. This is automatically done when using `train.ModelTrainer` or `bmdal.algorithms.select`, but we do not use these here.

In [2]:
import torch
torch.backends.cuda.matmul.allow_tf32 = False

## Feature maps and feature data

Kernels and their corresponding feature maps are central to our framework. The class `bmdal.feature_map.FeatureMap` represents both a feature map and the corresponding kernel. Sometimes, for example if the features are infinite-dimensional, it may not be able to compute the features but just the kernel values. As an input to feature maps, we use data represented by subclasses of `bmdal.feature_data.FeatureData`. The simplest type of data is `TensorFeatureData`, which simply represents a tensor:

In [3]:
from bmdal.feature_data import TensorFeatureData
import torch

data = TensorFeatureData(torch.arange(10)[:, None])
print(data.get_tensor())

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])


The first dimension of the tensor is interpreted as the batch dimension. We can index `FeatureData` objects along the batch dimension similar to torch Tensors or numpy arrays, but with a few less options and with a caveat:

In [4]:
print(data[2].get_tensor())
print(data[1:3].get_tensor())
print(data[torch.as_tensor([1, 2, 3])].get_tensor())

tensor([[2]])
tensor([[1],
        [2]])
tensor([[1],
        [2],
        [3]])


Note that the tensor returned by `data[2].get_tensor()` is still a rank-2 tensor (with shape `[1, 1]` instead of `[1]`). This behavior makes our implementation much easier. Note that the implementation does not support indexing with step sizes other than $1$:

In [5]:
print(data[::-1].get_tensor())

ValueError: Cannot handle slices with step size other than 1

In the error message, you can see that `TensorFeatureData` uses an `Indexes` object to represent the indexing. This is an internal class that is used a lot for indexing (for example when batching operations), but we don't need to worry about it for now.

We can now define a feature map for our one-dimensional feature data:

In [6]:
from bmdal.feature_maps import IdentityFeatureMap

feature_map = IdentityFeatureMap(n_features=1)

The constructed feature map corresponds to the simple identity feature map $\phi(x) = x$. A large variety of feature maps can be found in `bmdal/feature_maps.py`, for example feature maps corresponding to base kernels, and they can be combined for example using other feature maps for sum or product kernels. Feature maps have many useful functions, but these functions are conveniently used through the `Features` class, which combines feature maps and feature data.

## Features and transformations

The `Features` class combines a feature map and corresponding feature data. It provides methods for the functionalities of the feature map and the feature data. In the following, we will create two `Features` objects with the same feature map, representing the same kernel on artificial train and pool data, respectively:

In [11]:
from bmdal.features import Features

n_features = 1024
n_train = 100
n_pool = 1000
torch.manual_seed(1234)
train_data = TensorFeatureData(torch.randn(n_train, n_features))
pool_data = TensorFeatureData(torch.randn(n_pool, n_features))
feature_map = IdentityFeatureMap(n_features=n_features)

train_features = Features(feature_map, train_data)
pool_features = Features(feature_map, pool_data)
all_features = train_features.concat_with(pool_features)

Now, we can for example compute the kernel matrix, its diagonal, the feature matrix, or the matrix of squared kernel distances $d_k(x_i, x_j)^2 = k(x_i, x_i) + k(x_j, x_j) - 2k(x_i, x_j)$ on the training set:

In [12]:
print(train_features.get_kernel_matrix(train_features).shape)
print(train_features.get_kernel_matrix_diag().shape)
print(train_features.get_feature_matrix().shape)
print(train_features.get_sq_dists(train_features).shape)

torch.Size([100, 100])
torch.Size([100])
torch.Size([100, 1024])
torch.Size([100, 100])


In our algorithms, we mostly need to compute individual rows of the kernel matrix, which we can obtain using indexing as for the feature data:

In [13]:
print(pool_features[0].get_kernel_matrix(all_features).shape)

torch.Size([1, 1100])


As for the feature data, indexing with a single index does not remove the first dimension for simplicity of implementation. 

Note that the computations of kernel matrices etc. are vectorized. If the vectorized computation of an entire kernel matrix (or another quantity) might cause a RAM overflow, we can compute the kernel matrix in batches:

In [14]:
print(all_features.batched(128).get_kernel_matrix(all_features.batched(128)).shape)

torch.Size([1100, 1100])


The computation above computes $128 \times 128$ chunks of the kernel matrix in a vectorized fashion and then assembles them to the whole kernel matrix. To get an insight into how this works, we check the type of the transformed feature data:

In [15]:
print(all_features.batched(128).feature_data)

We see that the type of the feature data has changed from `TensorFeatureData` to `BatchedFeatureData` (which holds the `TensorFeatureData` object internally). Each `FeatureData` subclass has to provide an iterator over tuples of `(FeatureData, Indexes)` objects that are used during computation. `BatchedFeatureData` provides an iterator that proceeds in batches of the given size (except for the last batch, which might be smaller). Methods like `get_kernel_matrix()` in `FeatureMap` then use this iterator to compute the result in batches. Nonetheless, if we use `get_tensor()`, the full tensor is returned at once:

In [16]:
print(torch.all(all_features.feature_data.get_tensor() == all_features.batched(128).feature_data.get_tensor()))

tensor(True)


There are many more methods for convenience, for example the following ones:

In [17]:
print(all_features.get_n_features())
print(len(all_features))
print(all_features.get_device())

1024
1100
cpu


Besides the functions shown above, `Features` objects support a variety of transformations corresponding to the kernel transformations from our paper. For example, random projections (a.k.a. sketching) can be applied. In order to apply the same random projections to both train and pool features, we create a transformation on one of the two and then apply it to both:

In [18]:
n_random_projections=256
sketch_tfm = train_features.sketch_tfm(n_features=n_random_projections)
train_features_sketched = sketch_tfm(train_features)
pool_features_sketched = sketch_tfm(pool_features)
print(train_features_sketched.get_n_features())

256


Here, the `sketch_tfm` modifies the feature map of the features to apply a sketching matrix. The new feature map is computed during `train_features.sketch_tfm()`, and then the resulting `sketch_tfm` object simply replaces the feature map of the `Features` objects that it is applied to. Hence, it should only be applied to `Features` objects that contain the same feature map (and an analogous type of feature data). We can see that the type of feature map changed:

In [19]:
print(train_features_sketched.feature_map)
print(train_features_sketched.feature_map.tfms)
print(train_features_sketched.feature_map.feature_map)

In order to obtain the sketched features, the `SequentialFeatureMap` first applies the `IdentityFeatureMap` (which does nothing) and then a `LinearFeatureMap`, which applies the sketching matrix. 

The sketched features yield similar distances as the original features (all distances are similar here since the data has been drawn from a high-dimensional normal distribution):

In [20]:
print(train_features[:3].get_sq_dists(train_features[:3]))
print(train_features_sketched[:3].get_sq_dists(train_features_sketched[:3]))

tensor([[   0.0000, 2293.8921, 2072.6375],
        [2293.8921,    0.0000, 2237.8413],
        [2072.6375, 2237.8413,    0.0000]])
tensor([[1.2207e-04, 2.1194e+03, 1.9793e+03],
        [2.1194e+03, 0.0000e+00, 2.1576e+03],
        [1.9793e+03, 2.1576e+03, 1.2207e-04]])


In order to make the computation of quantities such as kernel matrices faster, we would like to apply the sketching matrix to the data once in advance. This is automatically done by the precompute transformation. We can (for the sake of example) also batch this operation and then afterwards call `.simplify()`, which assembles the batches back together such that subsequent operations will not be batched:

In [21]:
train_features_sketched_precomputed = train_features_sketched.batched(128).precompute().simplify()
pool_features_sketched_precomputed = pool_features_sketched.batched(128).precompute().simplify()

Another way to achieve the same result with a transform object would be to use `bmdal.features.PrecomputeTransform(128)`. We now see that we are back with an `IdentityFeatureMap` and `TensorFeatureData` again:

In [22]:
print(train_features_sketched_precomputed.feature_map)
print(train_features_sketched_precomputed.feature_data)

Crucially, `precompute()` does not change resulting quantities such as feature map, kernel matrix, or squared kernel distances (up to numerical precision):

In [23]:
print(train_features_sketched[:3].get_kernel_matrix(train_features_sketched[:3]))
print(train_features_sketched_precomputed[:3].get_kernel_matrix(train_features_sketched_precomputed[:3]))

tensor([[ 893.5628, -108.3986,  -57.4941],
        [-108.3986, 1009.0555,  -88.8848],
        [ -57.4941,  -88.8848,  970.7518]])
tensor([[ 893.5627, -108.3986,  -57.4940],
        [-108.3986, 1009.0555,  -88.8848],
        [ -57.4940,  -88.8848,  970.7517]])


After precomputation, subsequent computations are faster:

In [24]:
import utils
with utils.TimePrinter('precomputed'):
    pool_features_sketched_precomputed.get_kernel_matrix(pool_features_sketched_precomputed)
with utils.TimePrinter('not precomputed'):
    pool_features_sketched.get_kernel_matrix(pool_features_sketched)

Time for precomputed: 0.0102644s
Time for not precomputed: 0.0314671s


Note that the implementation of `precompute()` depends on the feature map. For example, the `ProductFeatureMap` corresponding to a product of kernels usually does not precompute the final feature matrix since it is more time- and memory-efficient to exploit the product structure.

There are of course more transformations corresponding to the kernel transformations in our paper, such as the posterior transformation or the acs-transformations. For an overview, we refer to `bmdal/algorithms.py` and `bmdal/features.py`.

## Implementing your own feature maps, feature data or transformations

If you want to implement your own components, please have a look at the documentation strings in `bmdal/feature_maps.py`, `bmdal/feature_data.py`, `bmdal/features.py`. The implementations of subclasses may serve as useful examples. Please be aware that some methods like `FeatureMap.get_kernel_matrix()` should not be overridden in subclasses directly since they perform an iteration to implement batching. Instead, corresponding methods like `FeatureMap.get_kernel_matrix_impl_()` should be overridden instead.

## Selection methods

Selection methods have a simpler interface. For example, MaxDist-TP can be used as follows:

In [25]:
from bmdal.selection import MaxDistSelectionMethod
sel_method = MaxDistSelectionMethod(pool_features=pool_features, train_features=train_features, sel_with_train=True)
new_idxs = sel_method.select(5)
print(new_idxs)

tensor([655, 228, 970, 508, 150])


Note that selection method objects have a state that gets updated during `select()`, hence `select()` can only be used once per `SelectionMethod` object. Most implemented selection methods inherit from `bmdal.selection.IterativeSelectionMethod`, which implements the iterative selection template from Appendix D.1 in our paper. If you want to implement your own selection method, you may want to take a look at the implementations in `bmdal/selection.py`.